# Atividade 3

In [3]:
import pandas as pd
from datetime import datetime
from siphon.simplewebservice.wyoming import WyomingUpperAir
import numpy as np

In [6]:
# Data para análise
date = datetime(2023, 7, 5, 12)
# local da análise
station = 'SBFL'
# Extraindo os dados
df = WyomingUpperAir.request_data(date, station) 
df.head() 

,pressure,height,temperature,dewpoint,direction,speed,u_wind,v_wind,station,station_number,time,latitude,longitude,elevation,pw
0,1021.0,5,17.6,13.9,0.0,0.0,-0.000000,-0.000000,SBFL,83899,2023-07-05 12:00:00,-27.67,-48.55,5.0,15.83
1,1000.0,182,16.4,12.0,345.0,14.0,3.623467,-13.522962,SBFL,83899,2023-07-05 12:00:00,-27.67,-48.55,5.0,15.83
2,957.0,555,13.8,9.9,319.0,17.0,11.153003,-12.830063,SBFL,83899,2023-07-05 12:00:00,-27.67,-48.55,5.0,15.83
3,950.8,610,13.9,9.4,315.0,17.0,12.020815,-12.020815,SBFL,83899,2023-07-05 12:00:00,-27.67,-48.55,5.0,15.83
4,925.0,842,14.2,7.2,330.0,25.0,12.500000,-21.650635,SBFL,83899,2023-07-05 12:00:00,-27.67,-48.55,5.0,15.83


## Função para determinar a classe de estabilidade de Pasquill

In [8]:
classificacao = pd.read_excel(r'C:\Users\vinicius\Documents\ENS5173_202401\inputs\Classes_Pasquill.xlsx')
classificacao

,Pasquill stability class,dT/dZ
0,A,dT/dZ < - 1.9
1,B,-1.9 ≤ dT/dZ < -1.7
2,C,-1.7 ≤ dT/dZ < -1.5
3,D,-1.5 ≤ dT/dZ < -0.5
4,E,-0.5 ≤ dT/dZ < 1.5
5,F,dT/dZ ≥ 1.5


In [9]:
def pasquill(temp, alt, i): 
    deltaT = np.diff(temp)
    deltaZ = np.diff(alt)
    dTdZ = (deltaT/deltaZ)*100 # Gradiente de temperatura; a cada 100 m  
    if dTdZ[i] < -1.9:
        classe = 'A'
    if -1.9 <= dTdZ[i] < -1.7:
        classe = 'B'
    if -1.7 <= dTdZ[i] < -1.5:
        classe = 'C'
    if -1.5 <= dTdZ[i] < -0.5:
        classe = 'D'
    if -0.5 <= dTdZ[i] < 1.5:
        classe = 'E'
    if dTdZ[i] >= 1.5:
        classe = 'F'
    return {'dTdZ': [dTdZ[i]], 'Classe de Pasquill': [classe]}

estabilidade = pd.DataFrame(data=pasquill(df['temperature'], df['height'], 0)) 
estabilidade 

,dTdZ,Classe de Pasquill
0,-0.677966,D


## Função de estimativa de coefiente de dispersão (sigmaYZ) para todas as classes de estabilidade

In [ ]:
u =np.rand
cn=
rad = 

In [10]:
# Função para estimar as dispersões lateral e vertical da pluma 
def sigmasYZ(UR, classe, x):
    if (classe=='A' or classe=='B') and UR=='urbano':
        sigmaY = 0.32*x*((1+0.0004*x)**(-0.5)) 
        sigmaZ = 0.24*x*((1+0.001*x)**(0.5)) 
    if classe=='C' and UR=='urbano':
        sigmaY = 0.22*x*((1+0.0004*x)**(-0.5)) 
        sigmaZ = 0.20*x 
    if classe=='D' and UR=='urbano':
        sigmaY = 0.16*x*((1+0.0004*x)**(-0.5)) 
        sigmaZ = 0.14*x*((1+0.0003*x)**(-0.5))
    if (classe=='E' or classe=='F') and UR=='urbano':
        sigmaY = 0.11*x*((1+0.0004*x)**(-0.5)) 
        sigmaZ = 0.08*x*((1+0.0015*x)**(-0.5)) 
    if classe=='A' and UR=='rural':
        sigmaY = 0.22*x*((1+0.0001*x)**(-0.5)) 
        sigmaZ = 0.20*x 
    if classe=='B' and UR=='rural':
        sigmaY = 0.16*x*((1+0.0001*x)**(-0.5))
        sigmaZ = 0.12*x      
    if classe=='C' and UR=='rural':
        sigmaY = 0.11*x*((1+0.0001*x)**(-0.5))
        sigmaZ = 0.08*x*((1+0.0002*x)**(-0.5))           
    if classe=='D' and UR=='rural':
        sigmaY = 0.08*x*((1+0.0001*x)**(-0.5))
        sigmaZ = 0.06*x*((1+0.0015*x)**(-0.5))      
    if classe=='E' and UR=='rural':
        sigmaY = 0.06*x*((1+0.0001*x)**(-0.5))
        sigmaZ = 0.03*x*((1+0.0003*x)**(-1))
    if classe=='F' and UR=='rural':
        sigmaY = 0.04*x*((1+0.0001*x)**(-0.5))
        sigmaZ = 0.016*x*((1+0.0003*x)**(-1))
    return {'Sigma Y': [sigmaY], 'Sigma Z': [sigmaZ]}

sigmas = pd.DataFrame(data=sigmasYZ('urbano', 'D', 100)) 
sigmas

,Sigma Y,Sigma Z
0,15.689291,13.79461


## Função de estimativa de sobrelevação da pluma
Métodos de Davidson-Bryant e de Holland. Deve ser considerado o efeito Tip-Downwash.

In [11]:
# Vs, u & Ts

## Função do modelo gaussiano

In [12]:
def modeloGaussiano(qs,sigmaY,sigmaZ,u,y,z,H):
    termo1 = qs/(2*np.pi*sigmaY*sigmaZ*u)
    termo2 = np.exp((-y**2)/(2*sigmaY**2))
    termo3 = np.exp((-(z-H)**2)/(2*sigmaZ**2)) + np.exp((-(z+H)**2)/(2*sigmaZ**2))
    conc = termo1*termo2*termo3
    conc = conc*10**6
    return conc # micro g/m3

## Simulações com o script
(Utilizando diferentes classes de estabilidade, velocidades do vento, alturas de chaminé. Considere a taxa de emissão que você estimou na primeira atividade. Encontre a altura de chaminé necessária para que as concentrações não violem os padrões da Resolução CONAMA 491).

## Figuras e discussão dos resultados